# Import Librairy

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from tqdm import tqdm
import matplotlib.pyplot as plt

# Download financial data

In [ ]:
def get_financial_data(company: list, period: str, interval: str):
    data = yf.download(tickers=' '.join(company),
                       period=period,
                       interval=interval,
                       group_by='columns',
                       auto_adjust=True,
                       prepost=True,
                       threads=True,
                       proxy=None)
    data = data.loc[:,data.columns.get_level_values(1).isin({"Close"})].droplevel(1, axis=1)
    return data

# Build Portfolio Optimization

In [ ]:
def sharpe_ratio_portfolio(data: pd.DataFrame, weight: np.array, risk_free_rate: float):
    log_return = np.log(1 + data.pct_change())
    expected_return = np.sum(log_return.mean() * weight * 252)
    expected_volatility = np.sqrt(
        np.dot(
            weight.T,
            np.dot(
                log_return.cov() * 252,
                weight
            )
        )
    )
    sharpe = (expected_return - risk_free_rate) / expected_volatility
    return expected_return, expected_volatility, sharpe

def MC_weight_sim(data: pd.DataFrame, risk_free_rate: float, N_sim: int, as_df: bool):
    size_of_portfolio = data.shape[1]
    sim_weights = np.zeros((N_sim, size_of_portfolio))
    sim_returns, sim_vol, sim_sharpe = np.zeros((N_sim, 1)), np.zeros((N_sim, 1)), np.zeros((N_sim, 1))
    for i in tqdm(range(N_sim)):
        weight = np.array(np.random.random(size_of_portfolio))
        exp_return, exp_vol, sharpe = sharpe_ratio_portfolio(data, weight, risk_free_rate)
        weights = weight / np.sum(weight)
        sim_weights[i, :] = weights
        sim_returns[i, :] = exp_return
        sim_vol[i, :] = exp_vol
        sim_sharpe[i, :] = sharpe
    global_result = np.concatenate((sim_weights, sim_returns, sim_vol, sim_sharpe), axis=1)
    if not as_df:
        return global_result
    else:
        col = [f"W{i + 1}" for i in range(size_of_portfolio)]
        col.extend(["Return", "Volatility", "Sharpe"])
        result_df = pd.DataFrame(
            data=global_result,
            columns=col
        )
        return result_df

def MC_sim_summary(data: pd.DataFrame, risk_free_rate: float, N_sim: int):
    all_sim = MC_weight_sim(data, risk_free_rate, N_sim, True)
    max_sharpe = all_sim.loc[all_sim["Sharpe"] == all_sim["Sharpe"].max()]
    min_volatility = all_sim.loc[all_sim["Volatility"] == all_sim["Volatility"].min()]
    print(max_sharpe)
    print(min_volatility)
    plt.scatter(all_sim["Sharpe"], all_sim["Volatility"])
    plt.show()

# Plot Simulation and show efficient frontier

In [ ]:
MC_sim_summary(get_financial_data(["SPY", "AAPL", "MSFT", "AMZN", "TSLA"], "1y", "1d"), 0.04, 10000)